In [25]:
import os
import pandas as pd
import GPUtil
import cpuinfo
import re
import time
import psutil
import pyperf

In [9]:
start = time.time()

gpu_dataset_path = 'data/gpu.csv'
cpu_dataset_path = 'data/cpu.csv'
regexp_cpu = '(Core|Ryzen).* (i\d-\d{3,5}.?|\d \d{3,5}.?)'
# need to write regex for gpu as well- to remove GPU brand name

In [10]:
import GPUtil
def detect_gpu():
    try:
        gpus = GPUtil.getGPUs()
        gpu_name = gpus[0].name
        print(f'GPU name: {gpu_name}')
        name=gpu_name.replace('NVIDIA ','')
        
#         print(name)
#         dirname = os.path.dirname(__file__)
#         relative_gpu_dataset_path = os.path.join(dirname, gpu_dataset_path)

        df = pd.read_csv("../data/gpu.csv")
    
        # it uses contains for more flexibility
        row = df[df['name'].str.contains(name)]
        if row.empty:
            # if gpu not found then leave standard TDP value
            print('GPU not found. Standard TDP= 250 assigned.')
        else:
            # otherwise assign gpu's TDP
            TDP = row.TDP.values[0]
            print(TDP)
    # ValueError arise when GPUtil can't communicate with the GPU driver
    except (ValueError, IndexError):
        # in case no GPU can be found
        print('[except] GPU not found. Standard TDP= 250 assigned.')

In [11]:
detect_gpu()

[except] GPU not found. Standard TDP= 250 assigned.


In [12]:
def detect_cpu():
    try:
        cpu_name = cpuinfo.get_cpu_info()['brand_raw']
        print(f'CPU_name: {cpu_name}')
        result = re.search(regexp_cpu, cpu_name)
        
        if result is not None:
            cpu_name = result.group(1) + ' ' + result.group(2)
       # print(cpu_name)
            
#         dirname = os.path.dirname(__file__)
#         relative_cpu_dataset_path = os.path.join(dirname, cpu_dataset_path)

        df = pd.read_csv("../data/cpu.csv")
    
        # it uses contains for more flexibility
        row = df[df['name'].str.contains(cpu_name)]
        if row.empty:
            # if gpu not found then leave standard TDP value
            print('CPU not found. Standard TDP=250 assigned.')
        else:
            # otherwise, assign CPU's TDP
            TDP = row.TDP.values[0]
            print(f'CPU recognized: TDP set to {row.TDP.values[0]}')

    except:
        # in case no CPU can be found
        print('[except] CPU not found. Standard TDP=250 assigned.')

In [29]:
detect_cpu()

CPU_name: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz
CPU recognized: TDP set to 25


In [22]:
# Get the number of processors in the system
num_processors = psutil.cpu_count()
print(num_processors)


total_power = psutil.cpu_percent() / 100.0

# Compute the average power per processor
avg_power = total_power / num_processors

print(f"Average power per processor: {avg_power:.2f} W")


8
Average power per processor: 0.01 W


In [30]:
# import psutil
# import os
# import energy

# # Get the current power consumption of the system
# power_consumption = psutil.sensors_battery().power_plugged

# # Get the energy consumption of the data center
# data_center_energy = energy.Energy('electricity', 1)

# # Calculate the PUE
# pue = data_center_energy.get_energy() / power_consumption

# # Print the result
# print(f"PUE: {pue}")

# Get the total power consumption of the system
total_power = psutil.sensors_battery().power_plugged or psutil.sensors_battery().power_now

# Get the power consumption of the IT equipment alone
it_power = psutil.cpu_percent() + psutil.virtual_memory().percent

# Calculate PUE
pue = total_power / it_power

print("PUE:", pue)

PUE: 0.010706638115631691


In [32]:
end = time.time()

Hours_time = (end-start)/(60*60)

Energy = (Hours_time*num_processors*avg_power*pue)/1000
print("Energy : ",Energy,"KWh")
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")

Energy :  3.7892573853631603e-07 KWh
The time of execution of above program is : 3033571.1982250214 ms
